**Imports**

In [1]:
library(tidyverse)

Warning message:
“package ‘tidyverse’ was built under R version 3.5.2”── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
Warning message:
“package ‘ggplot2’ was built under R version 3.5.2”Warning message:
“package ‘dplyr’ was built under R version 3.5.2”Warning message:
“package ‘stringr’ was built under R version 3.5.2”Warning message:
“package ‘forcats’ was built under R version 3.5.2”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()


# Fitted coefficients and difficulty preferences

In [2]:
f <- function() { 
    # Load parameter data
    df1 <- tbl_df(read.csv('data/model_results/param_fits_clean.csv')) %>%
      dplyr::filter(vars=='rpc,rlp', nam > 0) %>%
      dplyr::mutate(rpc_norm = rpc/sqrt(rpc^2+rlp^2), rlp_norm = rlp/sqrt(rpc^2+rlp^2)) #%>%
      # dplyr::mutate(wPC = (wPC - mean(wPC))/sd(wPC),
      #               wLP = (wLP - mean(wLP))/sd(wLP))

    # Load RelT data
    df2 <- tbl_df(read.csv('data/model_data.csv')) %>% 
      dplyr::select(sid, trial, abst1, abst2, abst3, abst4) %>%
      dplyr::mutate(A4minusA3 = abst4-abst3, A3minusA1 = abst3-abst1) %>%
      dplyr::filter(trial==250)

    # Join datasets
    df <- dplyr::inner_join(df1, df2, by='sid')
    remove(df1, df2)

    # Convert categorical variables to factors
    df <- dplyr::mutate_at(df, vars(sid, group, nam), funs(factor))

    # Set colors
    df$group <- recode(df$group, '0' = 'IG', '1' = 'EG')

#     # Fit linear models
#     df <- within(df, group <- relevel(group, ref = 'IG'))
#     lm_ <- lm(A3minusA1 ~ group*(rpc_norm+rlp_norm), data=df)
#     print(summary(lm_))
#     lm_ <- lm(A4minusA3 ~ group*(rpc_norm+rlp_norm), data=df)
#     print(summary(lm_))
    # Fit linear models
    df <- within(df, group <- relevel(group, ref = 'IG'))
    lm_ <- lm(rpc_norm ~ group*(A3minusA1+A4minusA3), data=df)
    print(summary(lm_))
    lm_ <- lm(rlp_norm ~ group*(A3minusA1+A4minusA3), data=df)
    print(summary(lm_))
}

f()

Warning message:
“funs() is soft deprecated as of dplyr 0.8.0
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once per session.”


Call:
lm(formula = rpc_norm ~ group * (A3minusA1 + A4minusA3), data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.24241 -0.33406 -0.04356  0.33188  1.52479 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)        0.0976458  0.0408726   2.389   0.0175 *  
groupEG           -0.0896801  0.0611854  -1.466   0.1437    
A3minusA1         -0.0055262  0.0004780 -11.560   <2e-16 ***
A4minusA3         -0.0057868  0.0006642  -8.712   <2e-16 ***
groupEG:A3minusA1 -0.0015969  0.0007961  -2.006   0.0457 *  
groupEG:A4minusA3  0.0008581  0.0008340   1.029   0.3043    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4903 on 314 degrees of freedom
Multiple R-squared:  0.5802,	Adjusted R-squared:  0.5735 
F-statistic:  86.8 on 5 and 314 DF,  p-value: < 2.2e-16


Call:
lm(formula = rlp_norm ~ group * (A3minusA1 + A4minusA3), data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.34